Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.766778
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.145408
Minibatch accuracy: 56.2%
Validation accuracy: 72.6%
Minibatch loss at step 100: 0.816857
Minibatch accuracy: 68.8%
Validation accuracy: 76.8%
Minibatch loss at step 150: 0.363143
Minibatch accuracy: 87.5%
Validation accuracy: 77.4%
Minibatch loss at step 200: 0.807926
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 250: 0.941471
Minibatch accuracy: 68.8%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.215978
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.459496
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.218808
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.967271
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 500: 0.634462
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
M

# Test with ELU (Faster than RELU)
Batch size changed from 16 to 128

In [19]:

batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.elu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [20]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.003078
Minibatch accuracy: 12.5%
Validation accuracy: 13.8%
Minibatch loss at step 50: 0.749019
Minibatch accuracy: 82.0%
Validation accuracy: 76.6%
Minibatch loss at step 100: 0.731732
Minibatch accuracy: 79.7%
Validation accuracy: 79.5%
Minibatch loss at step 150: 0.797420
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 200: 0.648261
Minibatch accuracy: 85.9%
Validation accuracy: 80.2%
Minibatch loss at step 250: 0.551041
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 300: 0.622786
Minibatch accuracy: 78.9%
Validation accuracy: 81.9%
Minibatch loss at step 350: 0.491006
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 400: 0.597648
Minibatch accuracy: 85.2%
Validation accuracy: 82.1%
Minibatch loss at step 450: 0.529020
Minibatch accuracy: 86.7%
Validation accuracy: 82.1%
Minibatch loss at step 500: 0.435512
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(hidden,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.893854
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.087864
Minibatch accuracy: 25.0%
Validation accuracy: 28.7%
Minibatch loss at step 100: 1.329527
Minibatch accuracy: 50.0%
Validation accuracy: 55.0%
Minibatch loss at step 150: 0.751715
Minibatch accuracy: 81.2%
Validation accuracy: 71.2%
Minibatch loss at step 200: 1.182177
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Minibatch loss at step 250: 1.117778
Minibatch accuracy: 68.8%
Validation accuracy: 76.7%
Minibatch loss at step 300: 0.373301
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.604092
Minibatch accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss at step 400: 0.172199
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.879360
Minibatch accuracy: 68.8%
Validation accuracy: 78.7%
Minibatch loss at step 500: 0.766709
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%


# Test with ELU
Batch size = 128

In [23]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layer1_biases)
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(hidden,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.elu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [24]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.140803
Minibatch accuracy: 10.9%
Validation accuracy: 12.5%
Minibatch loss at step 50: 0.839501
Minibatch accuracy: 77.3%
Validation accuracy: 71.6%
Minibatch loss at step 100: 0.737142
Minibatch accuracy: 78.9%
Validation accuracy: 78.7%
Minibatch loss at step 150: 0.893893
Minibatch accuracy: 74.2%
Validation accuracy: 79.7%
Minibatch loss at step 200: 0.738219
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 250: 0.520592
Minibatch accuracy: 88.3%
Validation accuracy: 80.8%
Minibatch loss at step 300: 0.645235
Minibatch accuracy: 75.0%
Validation accuracy: 81.5%
Minibatch loss at step 350: 0.468050
Minibatch accuracy: 85.9%
Validation accuracy: 82.0%
Minibatch loss at step 400: 0.650908
Minibatch accuracy: 80.5%
Validation accuracy: 82.2%
Minibatch loss at step 450: 0.545229
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 500: 0.394252
Minibatch accuracy: 90.6%
Validation accuracy: 83.5%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [47]:
import math as math
batch_size = 128
patch_size = 4
depth = 16
l1_size = 705
l2_size = 205


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layerConv1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layerConv1_biases = tf.Variable(tf.zeros([depth]))
  layerConv2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth*2], stddev=math.sqrt(2.0/(patch_size*patch_size*depth))))
  layerConv2_biases = tf.Variable(tf.zeros([depth*2]))
  layerConv3_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*2, depth*4], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*2))))
  layerConv3_biases = tf.Variable(tf.zeros([depth*4]))
  layerConv4_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*4, depth*4], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*4))))
  layerConv4_biases = tf.Variable(tf.zeros([depth*4]))
  layerConv5_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*4, depth*16], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*4))))
  layerConv5_biases = tf.Variable(tf.zeros([depth*16]))

  layer1_weights = tf.Variable(tf.truncated_normal(
            [image_size//7*image_size//7*(depth*4), l1_size], 
            stddev=math.sqrt(2.0/(image_size//7*image_size//7*(depth*4)))))
  layer1_biases = tf.Variable(tf.zeros([l1_size]))
  layer2_weights = tf.Variable(tf.truncated_normal(
            [l1_size, l2_size], stddev=math.sqrt(2.0/(l1_size))))
  layer2_biases = tf.Variable(tf.zeros([l2_size]))
  layer3_weights = tf.Variable(tf.truncated_normal(
            [l2_size, num_labels], stddev=math.sqrt(2.0/(l2_size))))
  layer3_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Model.
  def model(data, use_dropout=False):
    conv = tf.nn.conv2d(data, layerConv1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerConv2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv2_biases)

    conv = tf.nn.conv2d(hidden, layerConv3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv3_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerConv4_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv4_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
   
    conv = tf.nn.conv2d(pool, layerConv5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv5_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.elu(tf.matmul(reshape, layer1_weights) + layer1_biases)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    nn_hidden_layer = tf.matmul(hidden, layer2_weights) + layer2_biases
    hidden = tf.nn.elu(nn_hidden_layer)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    return tf.matmul(hidden, layer3_weights) + layer3_biases
  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 3000, 0.86, staircase=True)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [48]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.462052
Minibatch accuracy: 10.9%
Validation accuracy: 17.1%
Minibatch loss at step 50: 0.583806
Minibatch accuracy: 83.6%
Validation accuracy: 80.4%
Minibatch loss at step 100: 0.612285
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 150: 0.708149
Minibatch accuracy: 79.7%
Validation accuracy: 84.7%
Minibatch loss at step 200: 0.598113
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 250: 0.370525
Minibatch accuracy: 89.8%
Validation accuracy: 86.2%
Minibatch loss at step 300: 0.591497
Minibatch accuracy: 83.6%
Validation accuracy: 86.2%
Minibatch loss at step 350: 0.295489
Minibatch accuracy: 91.4%
Validation accuracy: 86.4%
Minibatch loss at step 400: 0.439530
Minibatch accuracy: 85.9%
Validation accuracy: 87.5%
Minibatch loss at step 450: 0.329455
Minibatch accuracy: 89.8%
Validation accuracy: 87.9%
Minibatch loss at step 500: 0.328938
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
M

In [49]:
import math as math
batch_size = 128
patch_size = 4
depth = 16
l1_size = 705
l2_size = 205


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layerConv1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layerConv1_biases = tf.Variable(tf.zeros([depth]))
  layerConv2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth*2], stddev=math.sqrt(2.0/(patch_size*patch_size*depth))))
  layerConv2_biases = tf.Variable(tf.zeros([depth*2]))
  layerConv3_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*2, depth*4], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*2))))
  layerConv3_biases = tf.Variable(tf.zeros([depth*4]))
  layerConv4_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*4, depth*4], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*4))))
  layerConv4_biases = tf.Variable(tf.zeros([depth*4]))
  layerConv5_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*4, depth*16], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*4))))
  layerConv5_biases = tf.Variable(tf.zeros([depth*16]))

  layer1_weights = tf.Variable(tf.truncated_normal(
            [image_size//7*image_size//7*(depth*4), l1_size], 
            stddev=math.sqrt(2.0/(image_size//7*image_size//7*(depth*4)))))
  layer1_biases = tf.Variable(tf.zeros([l1_size]))
  layer2_weights = tf.Variable(tf.truncated_normal(
            [l1_size, l2_size], stddev=math.sqrt(2.0/(l1_size))))
  layer2_biases = tf.Variable(tf.zeros([l2_size]))
  layer3_weights = tf.Variable(tf.truncated_normal(
            [l2_size, num_labels], stddev=math.sqrt(2.0/(l2_size))))
  layer3_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Model.
  def model(data, use_dropout=False):
    conv = tf.nn.conv2d(data, layerConv1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerConv2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv2_biases)

    conv = tf.nn.conv2d(hidden, layerConv3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv3_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerConv4_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv4_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
   
    conv = tf.nn.conv2d(pool, layerConv5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerConv5_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.elu(tf.matmul(reshape, layer1_weights) + layer1_biases)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    nn_hidden_layer = tf.matmul(hidden, layer2_weights) + layer2_biases
    hidden = tf.nn.elu(nn_hidden_layer)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    return tf.matmul(hidden, layer3_weights) + layer3_biases
  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 1000, 0.9999, staircase=True)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))



In [50]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.376995
Minibatch accuracy: 10.9%
Validation accuracy: 10.1%
Minibatch loss at step 50: 0.587227
Minibatch accuracy: 82.0%
Validation accuracy: 80.4%
Minibatch loss at step 100: 0.623894
Minibatch accuracy: 80.5%
Validation accuracy: 83.3%
Minibatch loss at step 150: 0.695412
Minibatch accuracy: 79.7%
Validation accuracy: 85.1%
Minibatch loss at step 200: 0.590059
Minibatch accuracy: 85.9%
Validation accuracy: 85.6%
Minibatch loss at step 250: 0.377872
Minibatch accuracy: 90.6%
Validation accuracy: 86.3%
Minibatch loss at step 300: 0.592968
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Minibatch loss at step 350: 0.322172
Minibatch accuracy: 91.4%
Validation accuracy: 86.5%
Minibatch loss at step 400: 0.464115
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
Minibatch loss at step 450: 0.381428
Minibatch accuracy: 91.4%
Validation accuracy: 87.3%
Minibatch loss at step 500: 0.324086
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
M

# Use Relu and add L2 regularization

In [9]:
import math as math
batch_size = 128
patch_size = 4
depth = 16
l1_size = 705
l2_size = 205


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layerConv1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layerConv1_biases = tf.Variable(tf.zeros([depth]))
  layerConv2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth*2], stddev=math.sqrt(2.0/(patch_size*patch_size*depth))))
  layerConv2_biases = tf.Variable(tf.zeros([depth*2]))
  layerConv3_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*2, depth*4], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*2))))
  layerConv3_biases = tf.Variable(tf.zeros([depth*4]))
  layerConv4_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*4, depth*4], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*4))))
  layerConv4_biases = tf.Variable(tf.zeros([depth*4]))
  layerConv5_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth*4, depth*16], stddev=math.sqrt(2.0/(patch_size*patch_size*depth*4))))
  layerConv5_biases = tf.Variable(tf.zeros([depth*16]))

  layer1_weights = tf.Variable(tf.truncated_normal(
            [image_size//7*image_size//7*(depth*4), l1_size], 
            stddev=math.sqrt(2.0/(image_size//7*image_size//7*(depth*4)))))
  layer1_biases = tf.Variable(tf.zeros([l1_size]))
  layer2_weights = tf.Variable(tf.truncated_normal(
            [l1_size, l2_size], stddev=math.sqrt(2.0/(l1_size))))
  layer2_biases = tf.Variable(tf.zeros([l2_size]))
  layer3_weights = tf.Variable(tf.truncated_normal(
            [l2_size, num_labels], stddev=math.sqrt(2.0/(l2_size))))
  layer3_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Model.
  def model(data, use_dropout=False):
    conv = tf.nn.conv2d(data, layerConv1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layerConv1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerConv2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layerConv2_biases)

    conv = tf.nn.conv2d(hidden, layerConv3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layerConv3_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerConv4_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layerConv4_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
   
    conv = tf.nn.conv2d(pool, layerConv5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layerConv5_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer1_weights) + layer1_biases)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    nn_hidden_layer = tf.matmul(hidden, layer2_weights) + layer2_biases
    hidden = tf.nn.elu(nn_hidden_layer)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    return tf.matmul(hidden, layer3_weights) + layer3_biases
  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(layerConv1_weights) + tf.nn.l2_loss(layerConv1_biases) +
                  tf.nn.l2_loss(layerConv2_weights) + tf.nn.l2_loss(layerConv2_biases) +
                  tf.nn.l2_loss(layerConv3_weights) + tf.nn.l2_loss(layerConv3_biases) +
                  tf.nn.l2_loss(layerConv4_weights) + tf.nn.l2_loss(layerConv4_biases) +
                  tf.nn.l2_loss(layerConv5_weights) + tf.nn.l2_loss(layerConv5_biases) +
                  tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                  tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                  tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)) 
  # Add the regularization term to the loss.
  loss += 1e-5 * regularizers
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.9999, staircase=True)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [10]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.334960
Minibatch accuracy: 7.8%
Validation accuracy: 9.9%
Minibatch loss at step 50: 1.007022
Minibatch accuracy: 74.2%
Validation accuracy: 59.5%
Minibatch loss at step 100: 0.856069
Minibatch accuracy: 76.6%
Validation accuracy: 80.5%
Minibatch loss at step 150: 0.718567
Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Minibatch loss at step 200: 0.607941
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 250: 0.387797
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Minibatch loss at step 300: 0.643311
Minibatch accuracy: 78.9%
Validation accuracy: 84.1%
Minibatch loss at step 350: 0.367151
Minibatch accuracy: 92.2%
Validation accuracy: 85.9%
Minibatch loss at step 400: 0.459973
Minibatch accuracy: 85.2%
Validation accuracy: 85.7%
Minibatch loss at step 450: 0.409168
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%
Minibatch loss at step 500: 0.348739
Minibatch accuracy: 87.5%
Validation accuracy: 87.1%
Min